# Derive Analytics on data using Python Jupyter Notebook, Watson Conversation & NAO Robot 
### <font color='blue'> Expose an integration point using websockets for orchestration with Node-RED using Watson Conversation & Novo Robot.</font>

## 1. Setup
To prepare your environment, you need to install some packages.

### 1.1 Install the necessary packages

You need the latest versions of these packages:<br>
- websocket-client: is a python client for the Websockets.<br>

** Install the websocket client: **

In [ ]:
!pip install websocket-client

### 1.2 Import packages and libraries

Import the packages and libraries that are needed:

In [ ]:
from io import StringIO
import requests
import json
import pandas as pd
import websocket
import json
from datetime import datetime
import thread
import time

## 2. Configuration

Add configurable items of the notebook below

### 2.1 Add your service credentials for Object Storage

You must create Object Storage service on Bluemix.
To access data in a file in Object Storage, you need the Object Storage authentication credentials.
Insert the Object Storage authentication credentials as <i><b>credentials_1</b></i> in the following cell after 
removing the current contents in the cell. 

# 3. Persistence and Storage

### 3.1 Configure Object Storage Client

In [ ]:
auth_url = credentials_1['auth_url']+"/v3"
container = credentials_1["container"]

IBM_Objectstorage_Connection = swiftclient.Connection(
    key=credentials_1['password'], authurl=auth_url, auth_version='3', os_options={
        "project_id": credentials_1['project_id'], "user_id": credentials_1['user_id'], "region_name": credentials_1['region']})

def create_container(container_name):
    """ Create a container on Object Storage.
    """
    x = IBM_Objectstorage_Connection.put_container(container_name)
    return x

def put_object(container_name, fname, contents, content_type):
    """ Write contents to Object Storage.
    """
    x = IBM_Objectstorage_Connection.put_object(
        container_name,
        fname,
        contents,
        content_type)
    return x

def get_object(container_name, fname):
    """ Retrieve contents from Object Storage.
    """
    Object_Store_file_details = IBM_Objectstorage_Connection.get_object(
        container_name, fname)
    return Object_Store_file_details[1]

## 4. Global Variable

#### Add global variable

In [ ]:
Data = 'Data.csv'

## 5. Read the Data & convert it into Dataframe

In [ ]:
'''Reading the dataset'''

df = pd.read_csv(StringIO(get_object(container, Data).decode('utf-8')))
df.head()

## 6. Create Generic re-usable functions for Statistics

#####      These functions can be applied to any dataset

In [ ]:
def get_max_val(tuple1,tuple2,tuple3,tuple4):
    '''This Function takes four input parameters to find max value'''
    mr = df[(df[tuple1[0]] == tuple1[1]) & (df[tuple2[0]] == tuple2[1]) & (df[tuple3[0]] == tuple3[1])] [tuple4[0]].max()
    return mr

In [ ]:
def get_min_val(tuple1,tuple2,tuple3,tuple4):
    '''This Function takes four input parameters to find min value'''
    mr = df[(df[tuple1[0]] == tuple1[1]) & (df[tuple2[0]] == tuple2[1]) & (df[tuple3[0]] == tuple3[1])] [tuple4[0]].min()
    return mr

In [ ]:
def get_total_val(tuple1,tuple2,tuple3):
    '''This Function takes three input parameters to find total value'''
    mr = df[(df[tuple1[0]] == tuple1[1]) & (df[tuple2[0]] == tuple2[1])] [tuple3[0]].sum()
    return mr

In [ ]:
def get_avg_val(tuple1,tuple2,tuple3):
    '''This Function takes three input parameters to find average value'''
    mr = df[(df[tuple1[0]] == tuple1[1]) & (df[tuple2[0]] == tuple2[1])] [tuple3[0]].mean()
    return mr

In [ ]:
def top_three(t1,t2):
    '''This Function takes two input parameters to find top three'''
    tt = df.groupby(t1)[[t2]].sum().apply(lambda x: x.sort_values(ascending=False).head(3))
    return tt

In [ ]:
def top_new(t1,t2,t3):
    '''This Function takes three input parameters to find top n value - top of n value is dynamic'''
    tt = df.groupby(t1)[[t2]].sum().apply(lambda x: x.sort_values(ascending=False).head(t3))
    return tt

In [ ]:
def top_three_1(t1,t2,t3,t4):
    '''This Function takes four input parameters to find top n value - top of n value is dynamic'''
    df1 = df[(df[t1[0]] == t1[1])] 
    tt = df1.groupby(t2)[[t3]].sum().apply(lambda x: x.sort_values(ascending=False).head(t4))
    return tt

In [ ]:
def top_three_2(t1,t2,t3,t4,t5):
    '''This Function takes five input parameters to find top n value - top of n value is dynamic'''
    df1 = df[(df[t1[0]] == t1[1]) & (df[t2[0]] == t2[1])] 
    tt = df1.groupby(t3)[[t4]].sum().apply(lambda x: x.sort_values(ascending=False).head(t5))
    return tt

In [ ]:
def Bottom_three(t1,t2):
    '''This Function takes two input parameters to find bottom three'''
    bt = df.groupby(t1)[[t2]].sum().apply(lambda x: x.sort_values(ascending=False).tail(3))
    return bt

In [ ]:
def Bottom_new(t1,t2,t3):
    '''This Function takes three input parameters to find bottom n value - bottom of n value is dynamic'''
    bt = df.groupby(t1)[[t2]].sum().apply(lambda x: x.sort_values(ascending=False).tail(t3))
    return bt

In [ ]:
def bottom_three_1(t1,t2,t3,t4):
    '''This Function takes four input parameters to find bottom n value - bottom of n value is dynamic'''
    df1 = df[(df[t1[0]] == t1[1])] 
    tt = df1.groupby(t2)[[t3]].sum().apply(lambda x: x.sort_values(ascending=False).tail(t4))
    return tt

In [ ]:
def bottom_three_2(t1,t2,t3,t4,t5):
    '''This Function takes five input parameters to find bottom n value - bottom of n value is dynamic'''
    df1 = df[(df[t1[0]] == t1[1]) & (df[t2[0]] == t2[1])] 
    tt = df1.groupby(t3)[[t4]].sum().apply(lambda x: x.sort_values(ascending=False).tail(t5))
    return tt

## 7. Expose Integration Point with a Websocket Client 

In [ ]:
'''This part of the code imports the question from websocket, executes the function as per 
   corresponding intent and sends the output back to the websocket.''' 


def on_message(ws, message):
    msg = json.loads(message)
    for item in msg["botresponse"]["messageout"]["context"]["entities_array"]:
        if item['entity'] == 'Year':
            date = datetime.strptime(item['value'],'%Y')
            item['value'] = date.year
    print msg  
        
    entitylist = []
    for index,entity in enumerate(msg["botresponse"]["messageout"]["context"]["entities_array"]):
        print(entity,index)
        entitylist.append((entity["entity"],entity["value"]))

    contextlist = [list(e) for e in msg["botresponse"]["messageout"]["context"].items()]

            
    if msg["botresponse"]["messageout"]["context"]["intent"] == 'summation':
            response = get_total_val(entitylist[1],entitylist[0],entitylist[2])
            response = round(response,2)
            print(response)
    elif msg["botresponse"]["messageout"]["context"]["intent"] == 'average':
            response = get_avg_val(entitylist[1],entitylist[0],entitylist[2])
            response = round(response,2)
            print(response)
    elif msg["botresponse"]["messageout"]["context"]["intent"] == 'max':
            response = get_max_val(entitylist[2],entitylist[1],entitylist[0],entitylist[3])
            response = round(response,2)
            print(response)
    elif msg["botresponse"]["messageout"]["context"]["intent"] == 'min':
            response = get_min_val(entitylist[2],entitylist[1],entitylist[0],entitylist[3])
            response = round(response,2)
            print(response)
    elif msg["botresponse"]["messageout"]["context"]["intent"] == 'top':
            response = top_new(entitylist[0][0],entitylist[2][0],entitylist[1][0])
            response = response.to_json()
            print(response)
    elif msg["botresponse"]["messageout"]["context"]["intent"] == 'bottom':
            response = Bottom_new(entitylist[0][0],entitylist[2][0],entitylist[1][0])
            response = response.to_json() 
            print(response)
            
    Question = str(msg["botresponse"]["messageout"]["input"]["text"])
    responsejson = {}
    responsejson["response"] = {Question:response}
    ws.send(json.dumps(responsejson))
        
def on_error(ws, error):
    print(error)

def on_close(ws):
    ws.send("DSX Listen End")

def on_open(ws):
    def run(*args):
        for i in range(10000):
            hbeat = '{"cmd":"DSX HeartBeat"}'
            ws.send(hbeat)
            time.sleep(100)

    thread.start_new_thread(run, ())
    
        
def start_websocket_listener():
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("ws://NODERED_BASE_URL/ws/ws-robosocket",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()

# 8. Start Websocket Client

In [ ]:
start_websocket_listener()